> To live! like a tree alone and free,
> and like a forest in solidarity...
> Nazim Hikmet

<center> <h1> Random Forests (Yes! No Forest Image) </h1> </center>



__Objectives__

- Introduction of 'bagging' procedure.

- Identifying the need for bootstrapping for decision trees.

- Comparing Random forests and bagging methods

- Evaluating a model by random forest model

# Review: Bootstrapping


<img src= "img/bootstrap1.png" style="height:250px">


# Bagging


Let's us one more time recall that if $Z_{1}, \cdots, Z_{n}$ are independent observations with variance $\sigma^{2}$ then the variance of the mean $\bar{Z}$ is given by $\frac{\sigma^{2}}{n}$. 

__How is this relevant now?__



We will use this idea calculate $$ \hat{f}^{1}(x), \cdots, \hat{f}^{B}(x)$$ where each $\hat{f}^{i}$ represents a decision tree fitted to the bootstrapped data.

Then we will make a prediction by: 

$$ \hat{f}_{\text{avg}}(x) = \frac{1}{B}\sum_{b=1}^{B} \hat{f}^{b}(x)$$

Note that this is for regression and for the classification we can get majority vote.

[sklearn averages over probabilities not majority vote](https://scikit-learn.org/stable/modules/ensemble.html#forest)


## Random Forests

__Problem__ We still have some problem with this approach and random forests will address this problem. Can you see the issue?

- If we have a strong predictor then this will dominate in each tree.

Hint: Correlated trees

## Sklearn for RandomForests

In [1]:
import pandas as pd

In [2]:
## you can download the data from -- https://www.kaggle.com/ishaanv/ISLR-Auto#Heart.csv

## or http://faculty.marshall.usc.edu/gareth-james/ISL/data.html
heart = pd.read_csv('data/Heart.csv', index_col = 0)
heart.head()
print(heart.shape)

(303, 14)


In [3]:
heart.dropna(axis= 0, how= 'any', inplace = True)
y = heart.AHD
heart.drop(columns= 'AHD', inplace = True)

In [4]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 1 to 302
Data columns (total 13 columns):
Age          297 non-null int64
Sex          297 non-null int64
ChestPain    297 non-null object
RestBP       297 non-null int64
Chol         297 non-null int64
Fbs          297 non-null int64
RestECG      297 non-null int64
MaxHR        297 non-null int64
ExAng        297 non-null int64
Oldpeak      297 non-null float64
Slope        297 non-null int64
Ca           297 non-null float64
Thal         297 non-null object
dtypes: float64(2), int64(9), object(2)
memory usage: 32.5+ KB


In [5]:
display(heart.Thal.value_counts())
display(heart.ChestPain.value_counts())

normal        164
reversable    115
fixed          18
Name: Thal, dtype: int64

asymptomatic    142
nonanginal       83
nontypical       49
typical          23
Name: ChestPain, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split 
X_train, X_test,y_train, y_test = train_test_split(heart, y, test_size= 0.20, stratify = y)

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
categorical_variables = X_train.select_dtypes(include=['object']).columns
numerical_variables = X_train.select_dtypes(include = ['int64', 'float64']).columns

In [9]:
import numpy as np

In [10]:
ohe = OneHotEncoder(drop = 'first')
X_categ = ohe.fit_transform(X_train[categorical_variables]).toarray()
X_num = X_train[numerical_variables].values
Xtrain = np.concatenate((X_categ, X_num), axis = -1,)
Xtrain.shape

(237, 16)

In [11]:
## now we should transform the test data 
## to be able to use it for the prediction

X_test_categ = ohe.transform(X_test[categorical_variables]).toarray()
X_test_num = X_test[numerical_variables].values
Xtest = np.concatenate((X_test_categ, X_test_num), axis = -1,)
Xtest.shape

(60, 16)

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
clf = RandomForestClassifier(n_estimators= 100, 
                             criterion= 'gini', 
                             max_features= 'auto',
                             oob_score= True)

__Your Turn__

- Use 5 fold cross_validation to fit random forest classifier we created above.
- Don't forget to return training scores and trained estimators.

In [19]:
from sklearn.model_selection import cross_validate

In [20]:
# %load -r 1-6 supplement.py
validator = cross_validate(clf,
                    Xtrain,
                    y_train,
                    return_train_score=True,
                    return_estimator=True,
                    cv=5)

__Your Turn__

- What is the type of validator above?

- Check test vs train(validation) scores.

- Print "mean +/- std" for both train and test scores

- Also print oob_scores and compare them with cross_validation scores

In [34]:
#%load -r 9-17 supplement.py

__Your Turn__

- Note that we have over-fitting problem. 

- Let's try to reduce over-fitting

In [50]:
# %load -r 20-27 supplement.py

### Extra: Pipelines?

In [18]:
## There is an "easier" way to do this
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [29]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first'))])

preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, numerical_variables),
        ('cat', categorical_transformer, categorical_variables)],)

rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators =100,
                                                            oob_score = True))])

In [58]:
pipe_validator = cross_validate(rf, 
                    X_train, 
                    y_train, 
                    return_train_score= True, 
                    return_estimator= True,
                    cv = 5)

In [31]:
## train scores
print(cv['train_score'])
## validation scores
print(cv['test_score'])

## let's pick one of the estimator for further investigation

est = cv['estimator'][0]

[1. 1. 1. 1. 1.]
[0.83333333 0.8125     0.70833333 0.78723404 0.86956522]


In [216]:
est['classifier'].oob_score_

0.6931216931216931

## Continue...

In [51]:
feature_importances = clf.feature_importances_

In [225]:
feature_importances

array([0.12017998, 0.02325638, 0.0992529 , 0.11423371, 0.01116945,
       0.02442466, 0.12333133, 0.07292237, 0.05854475, 0.05918206,
       0.09408099, 0.02741546, 0.00420568, 0.01360422, 0.10584749,
       0.04834858])

In [52]:
# be careful with the order of columns
columns =numerical_variables.tolist() + ohe.get_feature_names().tolist()

In [53]:
importances = pd.DataFrame(data= feature_importances, index = columns, columns= ['feature_importances'])

importances.sort_values(by = 'feature_importances', ascending = False)

,feature_importances
Chol,0.243241
x0_typical,0.150684
x1_reversable,0.150458
Fbs,0.128949
x0_nonanginal,0.118249
x1_normal,0.071028
x0_nontypical,0.038257
RestECG,0.031842
Oldpeak,0.024891
Age,0.013420


### Extra Material 

- [Sklearn averages probabilities in RF implementation](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [On the variance](https://newonlinecourses.science.psu.edu/stat414/node/167/)

- [Do RF immune to overfitting?](https://en.wikipedia.org/wiki/Talk%3ARandom_forest)

- [Tricky stuff with respect to feature importance](http://rnowling.github.io/machine/learning/2015/08/10/random-forest-bias.html)

- [An interesting implementation of feature importance](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances_faces.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-faces-py)

- [Different Ensemble Methods in sklearn](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [ISLR - section 8.2](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf)